# Programming Assignment: Implementing Clustering Validation Measures

## Cluster Analysis in Data Mining Course - Coursera

* Author: Michael Onishi
* Date: October 30th, 2019

##Description
In this assignment, you will be implementing two clustering validation measures: Normalized Mutual Information (NMI) and Jaccard similarity.

You will be given one ground-truth clustering (partition) results and five clustering test cases. You need to evaluate the clustering test cases with regard to the ground-truth by NMI and Jaccard measures and submit your measures. You will be graded based on whether your measures are correct.

The ground-truth clustering (partition) results are stored in file "partitions.txt"; the five clustering result test cases are stored in file "clustering_1.txt", ..., "clustering_5.txt".

All files including partitions.txt, clustering_1.txt, ..., can be downloaded from the data.zip file attached below.

Each clustering result (both ground-truth and test cases) is represented by a file. Each line in a file consists of two integers, separated by a space. The first integer represents the id of a data item, and the second integer represents the id of the cluster which this item belongs to.

You need to submit a file titled "scores.txt" consisting of 5 lines. Each line contains two float numbers separated by a space. The first number of the i-th line represents the NMI measure you calculated for the i-th test case i (i.e. "clustering_i.txt") with regard to the ground-truth given in "partitions.txt", and the second number of the i-th line represents the Jaccard measure you calculated for the i-th test case.

As an example, a valid submission may look like:

<pre>
0.1000000 0.2000000
0.3000000 0.4000000
0.5000000 0.6000000
0.7000000 0.8000000
0.9000000 1.0000000
</pre>

You will be graded based on whether your file format is correct and onhow many measures you submitted are correct.



In [0]:
! wget -O data.zip "https://d3c33hcgiwev3.cloudfront.net/_75cb9c8852d35d351cb44551fcd26d3a_data.zip?Expires=1572480000&Signature=QpJh7T~xz7k8jXn7ZRrxnc8mJpQMOCzq-dUREztP0w-aWKjw8CuDSo9QHUIIAbW5whFKBZUtmTnkXveEEbuFVTjPOvpNiVoHMwsTHvABM82dzMsxiS0juyonnxk~Xzr5rkpm6pCdusgzULbBsvMyxik7yYL2zmARk~UsK9LGNlw_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
! unzip data.zip

--2019-10-30 12:22:21--  https://d3c33hcgiwev3.cloudfront.net/_75cb9c8852d35d351cb44551fcd26d3a_data.zip?Expires=1572480000&Signature=QpJh7T~xz7k8jXn7ZRrxnc8mJpQMOCzq-dUREztP0w-aWKjw8CuDSo9QHUIIAbW5whFKBZUtmTnkXveEEbuFVTjPOvpNiVoHMwsTHvABM82dzMsxiS0juyonnxk~Xzr5rkpm6pCdusgzULbBsvMyxik7yYL2zmARk~UsK9LGNlw_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A
Resolving d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)... 13.32.86.189, 13.32.86.5, 13.32.86.3, ...
Connecting to d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)|13.32.86.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6776 (6.6K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   6.62K  --.-KB/s    in 0.001s  

2019-10-30 12:22:26 (5.84 MB/s) - ‘data.zip’ saved [6776/6776]

Archive:  data.zip
  inflating: clustering_1.txt        
   creating: __MACOSX/
  inflating: __MACOSX/._clustering_1.txt  
  inflating: clustering_2.txt        
  inflating: __MACOSX/._c

In [0]:
import pandas as pd
import numpy as np
import math

# number of combinations for calculation Jaccard
def nCr(n,r):
  if n-r == 0:
    return 1
  if n-r < 0:
    return 0
  f = math.factorial
  return f(n) // f(r) // f(n-r)

column_names = []
records = {}
with open('partitions.txt') as in_file:
  column_names.append(in_file.name[:-4])
  for line in in_file:
    r = line.strip().split()
    records[int(r[0])] = [int(r[1])]

for i in range(1, 6):
  with open(f'clustering_{i}.txt') as in_file:
    column_names.append(in_file.name[:-4])
    for line in in_file:
      r = line.strip().split()
      records[int(r[0])].append(int(r[1]))  

In [0]:
print(list(records.items())[:5])
print(column_names)
print(len(records))

[(1, [2, 2, 2, 2, 2, 0]), (2, [0, 0, 0, 0, 0, 0]), (3, [2, 2, 1, 1, 2, 0]), (4, [1, 1, 1, 1, 1, 1]), (5, [2, 2, 2, 2, 2, 0])]
['partitions', 'clustering_1', 'clustering_2', 'clustering_3', 'clustering_4', 'clustering_5']
300


In [0]:
N = len(records)
df = pd.DataFrame(data = records.values(), index = records.keys(), columns = column_names)

In [0]:
df.head()

,partitions,clustering_1,clustering_2,clustering_3,clustering_4,clustering_5
1,2,2,2,2,2,0
2,0,0,0,0,0,0
3,2,2,1,1,2,0
4,1,1,1,1,1,1
5,2,2,2,2,2,0


In [0]:
nt = df.partitions.value_counts()
nci = [(df[f'clustering_{i}'].value_counts()) for i in range(1,6)]
pt = (nt / N)
pci = [(nci[i] / N) for i in range(5)]

In [0]:
print(nt)
print(nci)

2    100
1    100
0    100
Name: partitions, dtype: int64
[2    100
1    100
0    100
Name: clustering_1, dtype: int64, 2    100
1    100
0    100
Name: clustering_2, dtype: int64, 2    100
1    100
0    100
Name: clustering_3, dtype: int64, 2    94
1    92
0    92
3    12
4    10
Name: clustering_4, dtype: int64, 0    200
1    100
Name: clustering_5, dtype: int64]


In [0]:
ans = []
ht = -(pt*np.log2(pt)).sum()
for k in range(1,6):
  ict = 0.0
  tp = 0.0
  fn = 0.0
  fp = 0.0

  for j in pt.index:
    fn += nCr(nt[j], 2)

    for i in pci[k-1].index:
      nij = len(df.query(f'partitions == {j} & clustering_{k} == {i}'))
      pij = nij / N
      #print(f'partitions == {j} & clustering_{k} == {i}', pij)
      if nij > 0:
        tp += nCr(nij, 2)
        ict += pij * np.log2(pij/(pci[k-1][i] * pt[j]))
  
  for i in nci[k-1].index:
    fp += nCr(nci[k-1][i], 2)

  fn -= tp
  fp -= tp
  jaccard = tp / (tp + fn + fp)
  hc = -(pci[k-1]*np.log2(pci[k-1])).sum()
  ans.append([ict/np.sqrt(ht * hc), jaccard])

In [0]:
ans

[[0.8896247665800591, 0.9116889804325438],
 [0.6456368113477221, 0.6794842795747569],
 [0.3915437463816927, 0.46493045279668543],
 [0.7677891606902613, 0.8005979461848434],
 [0.7611702597222877, 0.5975855130784709]]

In [0]:
with open('scores.txt', 'w') as out_file:
  for a in ans:
    out_file.write(" ".join(map(str,a)))
    out_file.write("\n")